In [ ]:
!pip install xgboost
!pip install mne

In [ ]:
import mne
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

# S104

In [ ]:
sub='S104'

In [ ]:
path='/content/gdrive/My Drive/WEEK1/'+sub+'/5_Files_for_model/'

eeg=mne.read_epochs(path+'epochs_eeg_2-epo.fif', preload=True)

digits_df = pd.read_csv(path+'Digits_trials_final.xlsx')
labels = digits_df['Digit_text'].values

In [ ]:
eeg.info

Извлекаем признаки из ЭЭГ

In [ ]:
n_epochs = len(eeg)
n_channels = eeg.info['nchan']
n_times = eeg.times.shape[0]

In [ ]:
gamma_features = np.zeros((n_epochs, n_channels * n_times))

for i in range(n_epochs):
    epoch_data = eeg[i].get_data()
    filtered_data = mne.filter.filter_data(epoch_data, sfreq=eeg.info['sfreq'], l_freq=60, h_freq=120)  # Фильтруем по гамма-ритму
    gamma_features[i, :] = filtered_data.flatten()  # Плоский массив для всех каналов

In [ ]:
pca = PCA(n_components=40)
gamma_features_pca = pca.fit_transform(gamma_features)

In [ ]:
X = gamma_features_pca
y = labels

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

### Кросс-валидация

In [ ]:
scores = cross_val_score(model, X, y, cv=4, scoring='accuracy')

In [ ]:
print("Средняя accuracy: %.2f%%" % (scores.mean() * 100))
print("Accuracy по фолдам:", scores)

### Без кросс-валидации

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:

param_grid = {
    'n_estimators': [100, 200, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 6, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=4, scoring='accuracy')

In [ ]:
grid_search.fit(X, y)

In [ ]:

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_index = grid_search.best_index_

print("Точность по фолдам для лучших параметров:")

for i in range(grid_search.cv):  # grid_search.cv - количество фолдов
    fold_accuracy = grid_search.cv_results_[f'split{i}_test_score'][best_index]
    print(f"Fold {i+1}: {fold_accuracy:.2f}")

mean_accuracy = grid_search.cv_results_['mean_test_score'][best_index]
print(f"\nСредняя точность для лучших параметров: {mean_accuracy:.2f}")

# S101

In [ ]:
sub = 'S101'

In [ ]:
path='/content/gdrive/My Drive/WEEK1/'+sub+'/5_Files_for_model/'

eeg=mne.read_epochs(path+'epochs_eeg_2-epo.fif', preload=True)

digits_df = pd.read_csv(path+'Digits_trials_final.xlsx')
labels = digits_df['Digit_text'].values

In [ ]:
labels

In [ ]:
n_epochs = len(eeg)
n_channels = eeg.info['nchan']
n_times = eeg.times.shape[0]

In [ ]:
gamma_features = np.zeros((n_epochs, n_channels * n_times))

for i in range(n_epochs):
    epoch_data = eeg[i].get_data()
    filtered_data = mne.filter.filter_data(epoch_data, sfreq=eeg.info['sfreq'], l_freq=60, h_freq=120)
    gamma_features[i, :] = filtered_data.flatten()

In [ ]:
pca = PCA(n_components=30)
gamma_features_pca = pca.fit_transform(gamma_features)

In [ ]:
gamma_features_pca

In [ ]:
X = gamma_features_pca


y = labels


In [ ]:
#model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', n_estimators = 300, learning_rate=0.3, max_depth=7)
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

### Кросс-валидация

In [ ]:
scores = cross_val_score(model, X, y, cv=4, scoring='accuracy')

In [ ]:
print("Средняя accuracy: %.2f%%" % (scores.mean() * 100))
print("Accuracy по фолдам:", scores)

### Без кросс-валидации

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
#Пааметры для подбора
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 10]
}

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=4, scoring='accuracy')

In [ ]:
grid_search.fit(X, y)

In [ ]:
#Лучшие параметры
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_index = grid_search.best_index_

print("Точность по фолдам для лучших параметров:")

for i in range(grid_search.cv):  # grid_search.cv - количество фолдов
    fold_accuracy = grid_search.cv_results_[f'split{i}_test_score'][best_index]
    print(f"Fold {i+1}: {fold_accuracy:.2f}")

mean_accuracy = grid_search.cv_results_['mean_test_score'][best_index]
print(f"\nСредняя точность для лучших параметров: {mean_accuracy:.2f}")

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

# S105

In [ ]:
sub='S105'

In [ ]:
path='/content/gdrive/My Drive/WEEK1/'+sub+'/5_Files_for_model/'

eeg=mne.read_epochs(path+'epochs_eeg_2-epo.fif', preload=True)

digits_df = pd.read_csv(path+'Digits_trials_final.xlsx')
labels = digits_df['Digit_text'].values

In [ ]:
n_epochs = len(eeg)
n_channels = eeg.info['nchan']
n_times = eeg.times.shape[0]

In [ ]:
gamma_features = np.zeros((n_epochs, n_channels * n_times))

for i in range(n_epochs):
    epoch_data = eeg[i].get_data()
    filtered_data = mne.filter.filter_data(epoch_data, sfreq=eeg.info['sfreq'], l_freq=60, h_freq=120)  # Фильтруем по гамма-ритму
    gamma_features[i, :] = filtered_data.flatten()  # Плоский массив для всех каналов

In [ ]:
pca = PCA(n_components=40)
gamma_features_pca = pca.fit_transform(gamma_features)

In [ ]:
X = gamma_features_pca

y = labels

In [ ]:
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

### CV


In [ ]:
scores = cross_val_score(model, X, y, cv=4, scoring='accuracy')

In [ ]:
print("Средняя accuracy: %.2f%%" % (scores.mean() * 100))
print("Accuracy по фолдам:", scores)

### сетка параметров

In [ ]:
#Пааметры для подбора
param_grid = {
    'n_estimators': [100, 200, 500],
    'learning_rate': [0.01, 0.1, 0.3],
    'max_depth': [3, 5, 10]
}

In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=4, scoring='accuracy')

In [ ]:
grid_search.fit(X, y)

In [ ]:
#Лучшие параметры
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

In [ ]:
best_index = grid_search.best_index_

print("Точность по фолдам для лучших параметров:")

for i in range(grid_search.cv):  # grid_search.cv - количество фолдов
    fold_accuracy = grid_search.cv_results_[f'split{i}_test_score'][best_index]
    print(f"Fold {i+1}: {fold_accuracy:.2f}")

mean_accuracy = grid_search.cv_results_['mean_test_score'][best_index]
print(f"\nСредняя точность для лучших параметров: {mean_accuracy:.2f}")